## Langchain (router chains)

* Router Chains são um tipo de chain que decide dinamicamente qual subchain executar, com base na entrada do usuário
* É como uma central inteligente de roteamento, que analisa a pergunta ou comando e escolhe qual caminho ou "ferramenta" deve ser usada.

In [1]:
# Bibliotecas

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [2]:
quimica_template = ChatPromptTemplate.from_template("""Você é um químico muito experiente.
Você é excelente em responder perguntas sobre química de forma clara e objetiva.
Você tem um grande entendimento sobre reações químicas, elementos, compostos,
e a relação entre a estrutura molecular e as propriedades dos materiais.
Quando você não sabe a resposta para uma pergunta, você admite que não sabe.

Aqui está uma pergunta: {input}""")

geografia_template = ChatPromptTemplate.from_template("""Você é um geógrafo muito bem informado.
Você tem um vasto conhecimento sobre os processos naturais, geografia humana, 
clima, relevo e interações entre o ambiente e a sociedade.
Você é habilidoso em explicar como fatores físicos e humanos afetam
o mundo ao nosso redor.

Aqui está uma pergunta: {input}""")

biologia_template = ChatPromptTemplate.from_template("""Você é um biólogo muito capacitado.
Você tem um grande conhecimento sobre os seres vivos, suas estruturas, funções
e a interação entre diferentes organismos e seus ambientes.
Você é excelente em explicar conceitos de biologia de maneira clara,
tanto para iniciantes quanto para estudantes avançados.

Aqui está uma pergunta: {input}""")

In [3]:
prompt_infos = [
    {
        "name": "Química",
        "description": "Ideal para responder pergunta de química",
        "prompt_template": quimica_template
    },
     {
        "name": "Geografia",
        "description": "Ideal para responder pergunta de geografia",
        "prompt_template": geografia_template
    },
      {
        "name": "Biologia",
        "description": "Ideal para responder pergunta de biologia",
        "prompt_template": biologia_template
    },
]

In [4]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
chains_destino = {}
for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info["prompt_template"], verbose=True)
    chains_destino[info["name"]] = chain
    
chains_destino

{'Química': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Você é um químico muito experiente.\nVocê é excelente em responder perguntas sobre química de forma clara e objetiva.\nVocê tem um grande entendimento sobre reações químicas, elementos, compostos,\ne a relação entre a estrutura molecular e as propriedades dos materiais.\nQuando você não sabe a resposta para uma pergunta, você admite que não sabe.\n\nAqui está uma pergunta: {input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x79771bc3bd10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x79771bc7d910>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='')),
 'Geografia': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTempla

In [6]:
destinos = [f'{p["name"]}: {p["description"]}' for p in prompt_infos]
destinos_str = "\n".join(destinos)
print(destinos_str)

Química: Ideal para responder pergunta de química
Geografia: Ideal para responder pergunta de geografia
Biologia: Ideal para responder pergunta de biologia


In [7]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinos_str
)

print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Química: Ideal para responder pergunta de química
Geografia: Ideal para responder pergunta de geograf

In [8]:
router_template = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(chat, router_template, verbose=True)

In [9]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destino,
    default_chain=default_chain,
    verbose=True
)

In [10]:
chain.invoke({"input": "O que é o El Niño?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Geografia: {'input': 'O que é o El Niño?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um geógrafo muito bem informado.
Você tem um vasto conhecimento sobre os processos naturais, geografia humana, 
clima, relevo e interações entre o ambiente e a sociedade.
Você é habilidoso em explicar como fatores físicos e humanos afetam
o mundo ao nosso redor.

Aqui está uma pergunta: O que é o El Niño?

> Finished chain.

> Finished chain.


{'input': 'O que é o El Niño?',
 'text': 'O El Niño é um fenômeno climático que ocorre de forma irregular e afeta principalmente a região do Oceano Pacífico. Ele é caracterizado pelo aquecimento anormal das águas superficiais do oceano, causando mudanças significativas nos padrões climáticos ao redor do mundo.\n\nDurante um evento de El Niño, há um enfraquecimento dos ventos alísios que normalmente sopram do leste para o oeste no Oceano Pacífico, o que resulta em um aumento da temperatura da superfície do mar. Isso pode provocar chuvas intensas em algumas regiões e secas em outras, afetando a agricultura, a pesca, os recursos hídricos e até mesmo o clima global.\n\nO El Niño também está associado a fenômenos meteorológicos extremos, como furacões, tempestades e ondas de calor, que podem ter impactos devastadores em diversas partes do mundo. É importante monitorar e compreender o El Niño para melhorar a previsão de eventos climáticos e minimizar seus efeitos sobre a sociedade e o meio a

In [11]:
chain.invoke({"input": "Pra que serve a tabela periódica?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Química: {'input': 'Pra que serve a tabela periódica?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um químico muito experiente.
Você é excelente em responder perguntas sobre química de forma clara e objetiva.
Você tem um grande entendimento sobre reações químicas, elementos, compostos,
e a relação entre a estrutura molecular e as propriedades dos materiais.
Quando você não sabe a resposta para uma pergunta, você admite que não sabe.

Aqui está uma pergunta: Pra que serve a tabela periódica?

> Finished chain.

> Finished chain.


{'input': 'Pra que serve a tabela periódica?',
 'text': 'A tabela periódica é uma ferramenta fundamental na química, pois organiza todos os elementos químicos conhecidos de acordo com suas propriedades e características. Ela ajuda os cientistas a prever o comportamento dos elementos, suas reações químicas, propriedades físicas e químicas, bem como suas afinidades eletroquímicas. A tabela periódica também fornece informações sobre a estrutura atômica dos elementos, seu número atômico, massa atômica, configuração eletrônica, entre outras informações essenciais. Em resumo, a tabela periódica é essencial para a compreensão e estudo da química.'}

In [12]:
chain.invoke({"input": "Pra que serve a fotossíntese?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Biologia: {'input': 'Pra que serve a fotossíntese?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um biólogo muito capacitado.
Você tem um grande conhecimento sobre os seres vivos, suas estruturas, funções
e a interação entre diferentes organismos e seus ambientes.
Você é excelente em explicar conceitos de biologia de maneira clara,
tanto para iniciantes quanto para estudantes avançados.

Aqui está uma pergunta: Pra que serve a fotossíntese?

> Finished chain.

> Finished chain.


{'input': 'Pra que serve a fotossíntese?',
 'text': 'A fotossíntese é um processo fundamental realizado pelas plantas, algas e algumas bactérias, onde a luz solar é convertida em energia química. Esse processo é essencial para a produção de alimentos e oxigênio na atmosfera. Durante a fotossíntese, as plantas absorvem dióxido de carbono do ar e água do solo, utilizando a energia da luz solar para transformar esses compostos em glicose e oxigênio. A glicose é utilizada como fonte de energia para o metabolismo celular das plantas, enquanto o oxigênio é liberado no ambiente, sendo essencial para a respiração de animais e outros organismos que dependem do oxigênio para sobreviver. Em resumo, a fotossíntese é responsável por fornecer energia para as plantas, produzir oxigênio e manter o equilíbrio do ciclo do carbono na natureza.'}